In [1]:
import pandas as pd
import numpy as np
import os, sys, regex

In [2]:
dala = os.getenv('HOME')

In [3]:
base_dir = f'{dala}/Documents/RunSheets/RunSheet3/Sheets'

In [4]:
cols1 = ['Sample_Name', 'Reporter', 'CT', 'Ct_Mean']

data = pd.DataFrame(columns=cols1)
data_list = []
for file in os.listdir(base_dir):
    if file.endswith('.xls'):
        data_list.append(file)
    pass
sheets = sorted(data_list)

In [5]:
len(np.unique(np.array(sheets)))

199

In [6]:
cols2 = ['Sample Name', 'Reporter', 'CT', 'Ct Mean', 'Cт', 'Cт Mean']

counts = 0

for sheet in sheets:
    df = pd.read_excel(f'{base_dir}/{sheet}')
    count2 = 0
#     print(sheet)
    for row in df.itertuples():
        count2 += 1
        if row[1] == 'Well':
            df = pd.read_excel(f'{base_dir}/{sheet}', 'Results', header=count2)
            cols3 = list(set(df.columns) & set(cols2))
#             print(cols3)
#             print(list(df.columns))
            df2 = df[sorted(cols3)]
            df2.columns = sorted(cols1)
            data = data.append(df2, ignore_index=True)
            counts += 1
            break
        else: pass

In [7]:
print(f'number of sheets in folder: {len(sheets)}')
print(f'data rows: {data.shape[0]}')
print(f'sheets processed: {counts}')

number of sheets in folder: 199
data rows: 50119
sheets processed: 199


In [8]:
data.head(15)

,Sample_Name,Reporter,CT,Ct_Mean
0,COVC 11289,FAM,Undetermined,NaN
1,COVC 11289,VIC,Undetermined,NaN
2,COVC 11289,CY5,31.572554,31.572554
3,COVC 11290,FAM,Undetermined,NaN
4,COVC 11290,VIC,Undetermined,NaN
5,COVC 11290,CY5,33.771969,33.771969
6,COVC 11291,FAM,Undetermined,NaN
7,COVC 11291,VIC,Undetermined,NaN
8,COVC 11291,CY5,31.526318,31.526318
9,COVC 11292,FAM,Undetermined,NaN


In [9]:
data2 = data[data['CT'] != 'Undetermined']

In [10]:
data3 = data2[data2['Reporter'].isin(['FAM', 'VIC', 'ABY']) == True]

In [11]:
data1 = data3[data3['Sample_Name'].isin(['PC', 'NC']) != True]

In [12]:
data4 = data1[data1.duplicated('Sample_Name', False) == True]

In [13]:
data4.head(10)

,Sample_Name,Reporter,CT,Ct_Mean
192,COVC 13583,FAM,37.328472,37.328472
193,COVC 13583,VIC,42.683975,42.683975
201,COVC 13586,FAM,35.068436,35.068436
202,COVC 13586,VIC,39.385445,39.385445
252,COVC 13603,FAM,27.376181,27.376181
253,COVC 13603,VIC,31.060112,31.060112
297,COVC 13618,FAM,32.132778,32.132778
298,COVC 13618,VIC,36.225323,36.225323
327,COVC 13628,FAM,33.411869,33.411869
328,COVC 13628,VIC,36.019318,36.019318


In [14]:
def cov_rename(x):
    try:
        if 'COV' not in x and '-R' in x:
            x = x.replace('-R', '')
        if 'COV' in x and 'COVC' not in x:
            x = x.replace('COV', 'COVC')
        int(x)
        x = str(x).replace(str(x), f'COVC{str(x)}')
    except (ValueError, AttributeError, TypeError): x = str(x).replace(' ', '')
    return x

In [15]:
regex

<module 'regex' from '/home/douso/.local/lib/python3.8/site-packages/regex/__init__.py'>

In [16]:
def cov_names(x):
    x = str(x).upper().strip()
    if len(x) == 8:
        x = x.replace('COVC', 'COVC0')
    elif len(x) == 7:
        x = x.replace('COVC', 'COVC00')
    elif len(x) == 6:
        x = x.replace('COVC', 'COVC000')
    elif len(x) == 5:
        x = x.replace('COVC', 'COVC0000')
    return x

In [17]:
data4['Reporter'].unique()

array(['FAM', 'VIC', 'ABY'], dtype=object)

In [18]:
data5 = data4.assign(Sample_Name=data4['Sample_Name'].apply(lambda x: cov_names(cov_rename(x))))

In [19]:
data5.head()

,Sample_Name,Reporter,CT,Ct_Mean
192,COVC13583,FAM,37.328472,37.328472
193,COVC13583,VIC,42.683975,42.683975
201,COVC13586,FAM,35.068436,35.068436
202,COVC13586,VIC,39.385445,39.385445
252,COVC13603,FAM,27.376181,27.376181


In [20]:
data6 = data5.sort_values(['Sample_Name', 'Reporter']).groupby('Sample_Name').mean()

In [21]:
data6#.head()

,Ct_Mean
Sample_Name,
COVC03200,17.061703
COVC03346,17.323146
COVC03349,18.884158
COVC03359,20.915926
COVC03377,16.579814
...,...
NAN,NaN
PC-EXTRACTION,37.241522
PC-KIT,30.432119


In [55]:
# data6.reset_index().to_excel(F'{base_dir}/cov-positiveCts.xlsx', index=False)

In [30]:
#data8.to_excel('cov-positivesCts.xlsx', index=False)

In [22]:
# biorepo = pd.read_excel('/home/douso/Documents/RunSheets/BioRepository/COVID -19 SAMPLE ARCHIVE.xlsx')

In [23]:
biorepo.head()

,Animal ID,Sample Type,Date of sample collection,Date Frozen,Volume aliquoted,Freezer No.,ALIQUOT A Sample ID,ALIQUOT A Storage box,Position in box,ALIQUOT B Sample ID,ALIQUOT B Storage box,Position in box.1,Aliquoted by,Owner,Experiment,Project,Compiled by,Project code,Unnamed: 18
0,Human,Respiratory in VTM,NaT,NaT,Approx 1.2ml,BSL3 Freezer,COVA00001,SARS0001,1A,COVB0001,SARS0002,1A,NaN,Samuel/Edward/MOH,SARS-CoV-2 rt RT-PCR,COVID Diagnosis,stephen,BMZ002501 - COVID,NaN
1,Human,Respiratory in VTM,NaT,NaT,Approx 1.2ml,BSL3 Freezer,COVA00002,SARS0001,2A,COVB0002,SARS0002,2A,NaN,Samuel/Edward/MOH,SARS-CoV-2 rt RT-PCR,COVID Diagnosis,stephen,BMZ002501 - COVID,NaN
2,Human,Respiratory in VTM,NaT,NaT,Approx 1.2ml,BSL3 Freezer,COVA00003,SARS0001,3A,COVB0003,SARS0002,3A,NaN,Samuel/Edward/MOH,SARS-CoV-2 rt RT-PCR,COVID Diagnosis,stephen,BMZ002501 - COVID,NaN
3,Human,Respiratory in VTM,NaT,NaT,Approx 1.2ml,BSL3 Freezer,COVA00004,SARS0001,4A,COVB0004,SARS0002,4A,NaN,Samuel/Edward/MOH,SARS-CoV-2 rt RT-PCR,COVID Diagnosis,stephen,BMZ002501 - COVID,NaN
4,Human,Respiratory in VTM,NaT,NaT,Approx 1.2ml,BSL3 Freezer,COVA00005,SARS0001,5A,COVB0005,SARS0002,5A,NaN,Samuel/Edward/MOH,SARS-CoV-2 rt RT-PCR,COVID Diagnosis,stephen,BMZ002501 - COVID,NaN


In [24]:
cols = ['ALIQUOT A Sample ID', 'ALIQUOT A Storage box', 'Position in box',
       'ALIQUOT B Sample ID', 'ALIQUOT B Storage box', 'Position in box.1']

In [25]:
biorepo_loc = biorepo[cols]

In [26]:
biorepo_loc.head()

,ALIQUOT A Sample ID,ALIQUOT A Storage box,Position in box,ALIQUOT B Sample ID,ALIQUOT B Storage box,Position in box.1
0,COVA00001,SARS0001,1A,COVB0001,SARS0002,1A
1,COVA00002,SARS0001,2A,COVB0002,SARS0002,2A
2,COVA00003,SARS0001,3A,COVB0003,SARS0002,3A
3,COVA00004,SARS0001,4A,COVB0004,SARS0002,4A
4,COVA00005,SARS0001,5A,COVB0005,SARS0002,5A


In [40]:
cts = pd.read_excel(f'{base_dir}/cov-positiveCts-dedup.xlsx', '30AndBelowCts')

In [30]:
df_rslt = pd.read_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged.xlsx')

In [31]:
df_rslt['RESULT'].unique()

array(['Negative', 'Positive', nan, 'neg', 'pos', 'negative', 'positive',
       'Neg', 'Pos', 'Inconclusive', 'Positve', 'Positive '], dtype=object)

In [ ]:
df_rslt_pos = df_rslt['']

In [41]:
cts.head()

,Sample_Name,Ct_Mean
0,COVC03966,16.018745
1,COVC03813,16.019527
2,COVC18190,16.104663
3,COVC17756,16.149453
4,COVC04305,16.190788


In [32]:
def cov_names(x):
    x = x.upper()
    if len(x) == 9:
        x = x.replace('COVC', 'COVA')
    elif len(x) == 8:
        x = x.replace('COVC', 'COVA0')
    elif len(x) == 7:
        x = x.replace('COVC', 'COVA00')
    elif len(x) == 6:
        x = x.replace('COVC', 'COVA000')
    elif len(x) == 5:
        x = x.replace('COVC', 'COVA0000')
    return x

In [33]:
def cov_namesA(x):
    x = str(x)
    if len(x) == 8:
        x = x.replace('COVA', 'COVA0')
    elif len(x) == 7:
        x = x.replace('COVA', 'COVA00')
    elif len(x) == 6:
        x = x.replace('COVA', 'COVA000')
    elif len(x) == 5:
        x = x.replace('COVA', 'COVA0000')
    elif len(x) == 4:
        x = x.replace('COVA', 'COVA00000')
    elif len(x) == 3:
        x = x.replace('COVA', 'COVA000000')
    elif len(x) == 2:
        x = x.replace('COVA', 'COVA0000000')
    return x

In [42]:
cts_cln = cts.assign(Sample_NameB=cts['Sample_Name'].apply(lambda x: cov_names(x)))

In [43]:
cts_cln

,Sample_Name,Ct_Mean,Sample_NameB
0,COVC03966,16.018745,COVA03966
1,COVC03813,16.019527,COVA03813
2,COVC18190,16.104663,COVA18190
3,COVC17756,16.149453,COVA17756
4,COVC04305,16.190788,COVA04305
...,...,...,...
992,COVC17136,30.971293,COVA17136
993,COVC21494,30.974595,COVA21494
994,COVC15500,30.974725,COVA15500
995,COVC20819,30.982569,COVA20819


In [36]:
biorepo_loc_cln = biorepo_loc.assign(ALIQUOT_A_Sample_ID=biorepo_loc['ALIQUOT A Sample ID'].apply(lambda x: cov_namesA(x)))

In [37]:
biorepo_loc_cln.head()

,ALIQUOT A Sample ID,ALIQUOT A Storage box,Position in box,ALIQUOT B Sample ID,ALIQUOT B Storage box,Position in box.1,ALIQUOT_A_Sample_ID
0,COVA00001,SARS0001,1A,COVB0001,SARS0002,1A,COVA00001
1,COVA00002,SARS0001,2A,COVB0002,SARS0002,2A,COVA00002
2,COVA00003,SARS0001,3A,COVB0003,SARS0002,3A,COVA00003
3,COVA00004,SARS0001,4A,COVB0004,SARS0002,4A,COVA00004
4,COVA00005,SARS0001,5A,COVB0005,SARS0002,5A,COVA00005


In [38]:
# biorepo_loc_cln.to_excel('/home/douso/Documents/RunSheets/BioRepository/COVID-19-Sample-Archive-loc-cln.xlsx')

In [44]:
cts_cln.head()

,Sample_Name,Ct_Mean,Sample_NameB
0,COVC03966,16.018745,COVA03966
1,COVC03813,16.019527,COVA03813
2,COVC18190,16.104663,COVA18190
3,COVC17756,16.149453,COVA17756
4,COVC04305,16.190788,COVA04305


In [45]:
cts_biorepo = cts_cln.set_index('Sample_NameB').merge(biorepo_loc_cln.set_index('ALIQUOT_A_Sample_ID'), how='inner', left_index=True, right_index=True)

In [46]:
cts_biorepo.head()

,Sample_Name,Ct_Mean,ALIQUOT A Sample ID,ALIQUOT A Storage box,Position in box,ALIQUOT B Sample ID,ALIQUOT B Storage box,Position in box.1
COVA03200,COVC03200,17.061703,COVA3200,SARS0065,4A,COVB3200,SARS0066,4A
COVA03346,COVC03346,17.323146,COVA3346,SARS0067,50E,COVB3346,SARS0068,50E
COVA03349,COVC03349,18.884158,COVA3349,SARS0067,53F,COVB3349,SARS0068,53F
COVA03359,COVC03359,20.915926,COVA3359,SARS0067,63G,COVB3359,SARS0068,63G
COVA03377,COVC03377,16.579814,COVA3377,SARS0067,81I,COVB3377,SARS0068,81I


In [48]:
seq_all = pd.read_excel('/home/douso/Documents/RunSheets/all-(un)sequenced-covs.xlsx')

In [49]:
seq_all.head()

,covcs
0,COVC13064
1,COVC13125
2,COVC14272
3,COVC14288
4,COVC14304


In [50]:
sequenced = list(seq_all['covcs'].apply(lambda x: cov_names(x)))

In [51]:
cts_biorepo_srt = (cts_biorepo.reset_index().sort_values('index'))

In [52]:
cts_biorepo_srt

,index,Sample_Name,Ct_Mean,ALIQUOT A Sample ID,ALIQUOT A Storage box,Position in box,ALIQUOT B Sample ID,ALIQUOT B Storage box,Position in box.1
0,COVA03200,COVC03200,17.061703,COVA3200,SARS0065,4A,COVB3200,SARS0066,4A
1,COVA03346,COVC03346,17.323146,COVA3346,SARS0067,50E,COVB3346,SARS0068,50E
2,COVA03349,COVC03349,18.884158,COVA3349,SARS0067,53F,COVB3349,SARS0068,53F
3,COVA03359,COVC03359,20.915926,COVA3359,SARS0067,63G,COVB3359,SARS0068,63G
4,COVA03377,COVC03377,16.579814,COVA3377,SARS0067,81I,COVB3377,SARS0068,81I
...,...,...,...,...,...,...,...,...,...
912,COVA22434,COVC22434,29.439827,COVA22434,SARS0499,21C,COVB22434,SARS0500,21C
913,COVA22479,COVC22479,17.455937,COVA22479,SARS0499,66G,COVB22479,SARS0500,66G
914,COVA22488,COVC22488,29.679459,COVA22488,SARS0499,75H,COVB22488,SARS0500,75H
915,COVA22541,COVC22541,23.465435,COVA22541,SARS0501,28C,COVB22541,SARS0502,28C


In [53]:
final = cts_biorepo_srt[cts_biorepo_srt['index'].isin(sequenced) == False].sort_values(['index', 'Ct_Mean']).drop_duplicates('index', keep='last')

In [54]:
final#.head()

,index,Sample_Name,Ct_Mean,ALIQUOT A Sample ID,ALIQUOT A Storage box,Position in box,ALIQUOT B Sample ID,ALIQUOT B Storage box,Position in box.1
0,COVA03200,COVC03200,17.061703,COVA3200,SARS0065,4A,COVB3200,SARS0066,4A
1,COVA03346,COVC03346,17.323146,COVA3346,SARS0067,50E,COVB3346,SARS0068,50E
2,COVA03349,COVC03349,18.884158,COVA3349,SARS0067,53F,COVB3349,SARS0068,53F
3,COVA03359,COVC03359,20.915926,COVA3359,SARS0067,63G,COVB3359,SARS0068,63G
4,COVA03377,COVC03377,16.579814,COVA3377,SARS0067,81I,COVB3377,SARS0068,81I
...,...,...,...,...,...,...,...,...,...
907,COVA21485,COVC21485,28.155712,COVA21485,SARS0477,39D,COVB21485,SARS0478,39D
908,COVA21489,COVC21489,26.502728,COVA21489,SARS0477,43E,COVB21489,SARS0478,43E
909,COVA21491,COVC21491,27.306566,COVA21491,SARS0477,45E,COVB21491,SARS0478,45E
910,COVA21494,COVC21494,30.974595,COVA21494,SARS0477,48E,COVB21494,SARS0478,48E


In [55]:
final.to_excel('/home/douso/Documents/RunSheets/BioRepository/cov-positiveCtsLocsIII.xlsx', index=False)